# このドキュメントの目的

半手動で行うテキストへのタグ付けプロセスを、その手順を含めてまとめること。

# 必要な環境情報

- MacOS
- python3.6
- anaconda
- mecab-python3

# 手順の概要

### オペレーション_その１

1. はじめにドキュメントをファイルに分けて用意します。
2. 読み込んだデータをMeCabで形態素解析して品詞分解する。(この品詞分解には独自にカスタマイズしたモジュールを使っています)
3. 2で検出した単語（＝品詞の自然活用系（ ×:飲める → ○：飲む ））を、スペース区切り文字列（わかち書き）に変換。
4. 3の結果をsklearnのtopic_modelを実行します。
5. 各ドキュメントの予測トピックカテゴリをファイルとして出力します。

### オペレーション_その2

1. オペレーション_その１で出力されたテキストのカテゴライズデータをエクセルで見ながら、手動でtagをつけたファイルを作成します。（半手動部分）
2. この時、全てのテキストにtagをつける必要はありません。パッと見て判断がついたものだけにします。
3. 手動でtagづけされたデータを再学習します。
4. まだtagを手動でつけていないテキストも含めて、tagを予測します。
5. 予測タグのデータがファイル出力されますので、間違った予測タグを修正します。
6. 修正したファイルを、3に入力し直して、もう一度再学習します。
7. 以下、3~6を繰り返して、精度を改善します。


In [ ]:
from module.mecab_wrapper import keitaiso
import os
import re


In [ ]:
# ドキュメントファイルの読み込み
DIR_TEXT_FILE = 'data/text/topic-news'
doc_files = [os.path.join(DIR_TEXT_FILE, f) for f in os.listdir(DIR_TEXT_FILE) if re.match('^topic-news-.*.txt$', f)]
doc_texts = list()

# テキストの前処理関数を定義する
def remove_top2line(text):
    #　初めの2行は記事のURLと掲載開始日時なので省く
    text = ' '.join(text.split('\n')[2:])    
    return text
    
def text_cleaning(text):
    transform_dict = {',':'、', '\t':' '}
    for f,t in transform_dict.items():
        text.replace(f, t)
    return text

for df in doc_files:
    with open(df, 'r') as f:
        text = f.read()
        text = remove_top2line(text)
        text = text_cleaning(text)
        doc_texts.append(text)

# 中身の確認
doc_files[0]
print( doc_texts[0] )

In [ ]:
# 自作のmecab wrapperで形態素解析の設定を行う。
# この形態素解析のwrapperでは、動詞は自動的に自然活用形に変換される。

kei = keitaiso(use_PoW=['名詞'], stop_words=['彼女', '彼', '私', 'する', '*'])

# 以下、実行サンプル
kei.tokenize(doc_texts[0])

In [ ]:
# doc_textsをスペース区切り文字列に変換する。
convert_wakachi = lambda text:  ' '.join([token[2] for token in kei.tokenize(text)])
doc_wakachis = [convert_wakachi(text) for text in doc_texts]

doc_wakachis[0]

In [ ]:
# doc - word matrix　を作成する（bags of words, 単なる出現数カウント）
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer =  CountVectorizer(
    max_df=0.50, # 50%以上のdocに含まれる単語は無視する。
    min_df=0.05, # 5%以下のdocにしか含まれない単語は無視する。    
    )
doc_word_matrix = count_vectorizer.fit_transform(doc_wakachis)

# 学習済みの　count_vectorizer はpickleで保存しておく。
import pickle
file_name = 'data/processed/count_vectorizer.pickle'
with open(file_name, 'wb') as f:
    pickle.dump(count_vectorizer, f)


In [ ]:
# topic model (LatentDirichletAllocation) のモジュールを読み込む
from sklearn.decomposition import LatentDirichletAllocation

N_TOPIC = 3
lda = LatentDirichletAllocation(n_components=N_TOPIC, random_state=0)

# 学習して保存しておく。
lda.fit(doc_word_matrix)
file_name = 'data/processed/lda.pickle'
with open(file_name, 'wb') as f:
    pickle.dump(lda, f)

# 分類された topic
import pandas as pd
df_doc_topic = pd.DataFrame(
    lda.transform(doc_word_matrix), 
    columns=['topic%02d'%i for i in range(N_TOPIC)]
)
df_doc_topic['doc_file'] = doc_files
df_doc_topic['doc_text'] = doc_texts

df_doc_topic

In [ ]:
# df_doc_topic をファイル出力する。
file_name = 'data/df_doc_topic.csv'
df_doc_topic.to_csv(file_name, index=False)



# 以上で、オペレーション_01 は終了です


# 続いて、オペレーション_02 を行います


In [ ]:
# 手続き_02で、手動でtagをつける作業を行います。
# tagをつけたデータを読み込みます。
# 'data/tag.txt' に手動でタグをつけたデータを読み込みます。
TAG_FILE = 'data/tag.txt'
try:
    df_tag = pd.read_csv(TAG_FILE, sep='\t')
except:
    df_tag = pd.read_csv(TAG_FILE, sep='\t', encoding="932")

# このように手動でtagをつけたものです。
# 全てのtextにtagをつける必要はありません。
df_tag


In [ ]:
# 後半のほとんどが未記入です。
df_tag[100:]

In [ ]:
# ここで、textからベクトルを生成する関数を定義しておきます。

text = '''
衆議院の総選挙が開始しました。
事前の電話調査によれば、与党、野党ともに指示が割れていて予断ができない状態です。
'''

## テキストをtopicに変換するために必要な学習済みインスタンスを読み込む。
file_name = 'data/processed/count_vectorizer.pickle'
with open(file_name, 'rb') as f:
    count_vectorizer = pickle.load(f)
file_name = 'data/processed/lda.pickle'
with open(file_name, 'rb') as f:
    lda = pickle.load(f)

## text_vectorize関数を作成し、textをtopicベクトルに変換する手続きを定義する。
def text_vectorize(text):
    _text = text_cleaning(text)
    token = kei.tokenize(_text)
    wakachi = ' '.join([to[2] for to in token])
    count_vec = count_vectorizer.transform([wakachi])[0]
    topic_vec = lda.transform(count_vec)
    # 今回は、topic_vecとcount_vec を変数とする　（tf-idfでもよい）
    return np.concatenate([topic_vec, count_vec.toarray()], axis=1)[0]

## このように、入力されたtextをtopicベクトルとして出力することができるようになりました。
x = text_vectorize(text)
x



In [ ]:
# 手動でつけたタグを学習します。
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

estimater_dict = dict()
tags    = [col for col in df_tag.columns if re.match('^tag_', col)] 
topics = [col for col in df_tag.columns if re.match('^topic', col)] 
for tag in tags:
    # tag を未記入の部分は学習から除外します
    train_index = np.logical_not(np.isnan(df_tag[tag]))
    y = df_tag.loc[train_index, tag]
    # 上記で定義したベクトル化関数を使います。
    X = [text_vectorize(text) for text in df_tag.loc[train_index, 'doc_text']]
    estimator = GradientBoostingClassifier()
    estimator.fit(X, y)
    estimater_dict[tag] = estimator

# タグごとの学習済みセットが出来上がります。
estimater_dict

In [ ]:
# では新しいtextに対して予測させてみます。
text = '''
衆議院の総選挙が開始しました。
事前の電話調査によれば、与党、野党ともに指示が割れていて予断ができない状態です。
'''
def predict(text):
    x = text_vectorize(text)
    predict_result = {}
    for tag, estimator in estimater_dict.items():
        predict_result[tag] = estimator.predict_proba([x])[0][1]

    return predict_result

predict(text) # 政治は予測できているようです。

In [ ]:
text = '''
先日、サッカーの世界大会がありました。
日本代表チームは予選突破し、決勝トーナメントへ駒を進めました。しかし、惜しくも３回戦で敗退しました。
次は五輪オリンピック予選に向けて、調整に入ります。
'''
predict(text) # 海外は予測できているけれど、スポーツはまだ不十分です。

In [ ]:
# より予測精度を上げるためには、1)モデルの改善、2)データの改善を行います。
# ここではデータの改善に取り組んでみます。　モデルはいつでも変更できますが、tagつきデータはどこにもありません。

# まずは、現状の予測 数値を　df_tag に追加します。
for index, row in df_tag.iterrows():
    predicted_dict = predict(row['doc_text'])
    for tag, val in predicted_dict.items():
        df_tag.loc[index, 'predict_'+tag] = val

df_tag

In [ ]:
# 予測は、　tag づけしていないデータにもつきます。
df_tag[100:]

In [ ]:
# ファイルとして出力します。
file_name = 'data/tag_predict.csv'
df_tag.to_csv(file_name, index=False)


# 後は、、、

上記で出力された　data/tag_predict.csv　を目視で確認して、予測が間違っている部分の　tag を手動で入力し直しましょう。

（予測が正しい場合を、再入力してもよいですが、あまり効果はありません。現状の学習の強化になるだけです）

入力し直したデータを、　data/tag_predict.txt に上書きします。

そして、また再学習することで精度が強化されていきます。

こうして、「正しいデータを、効率的に増やしていきます」

後は、学習モデルをより高度なものに変更してみてもよいでしょう。

RNNでもBERTなど、自然言語の判別アルゴリズムは2019年の現在では頻繁にアップデートされています。

それらは、tagづけされたデータが出来上がってから改めて再学習することができます。